In [700]:
import mpg.ml.model as mpg_model
import mpg.ml.model.mlp as mlp
import mpg.ml.model.gnn as gnn
import mpg.gnn.aggregator as agg
import mpg.gnn.gcn as gcn
import importlib
import tensorflow as tf
importlib.reload(mlp)
importlib.reload(gnn)
importlib.reload(gcn)
importlib.reload(agg)

<module 'mpg.gnn.aggregator' from '/home/ramizouari/Academic/AI/MeanPayOffGames/notebooks/mpg/gnn/aggregator.py'>

In [701]:
n=20
p=0.3
batches=16
A=tf.cast(tf.random.uniform([batches,n,n]) > p,dtype=tf.float32)
W=tf.random.uniform([batches,n,n])
G=tf.stack([A,W],axis=-1)
X=tf.random.uniform([batches])
P=tf.math.softmax(tf.random.uniform([batches,n]),axis=-1)
V=tf.math.sign(tf.random.uniform([batches])-0.5)



model=mlp.MLP(graph_size=n,num_actions=n,state_shape=())
gnn_model=gnn.GNN()

(None, None)


2023-06-15 03:08:24.484074: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder' with dtype float and shape [?]
	 [[{{node Placeholder}}]]


In [702]:
gnn_model.compile(loss={"policy_targets":"categorical_crossentropy", "value_targets":"mean_squared_error"},
                           optimizer=tf.keras.optimizers.Adam())


In [703]:
inputs={"environment":G,"state":X}
outputs={"policy_targets":P,"value_targets":V}
gnn_model.fit(inputs, outputs, batch_size=8, epochs=3, verbose=1)

Epoch 1/3
2/2 [==============================] - 2s 10ms/step - loss: 4.1033 - value_targets_loss: 1.1058 - policy_targets_loss: 2.9975
Epoch 2/3
2/2 [==============================] - 0s 13ms/step - loss: 4.0306 - value_targets_loss: 1.0331 - policy_targets_loss: 2.9975
Epoch 3/3
2/2 [==============================] - 0s 12ms/step - loss: 3.9139 - value_targets_loss: 0.9164 - policy_targets_loss: 2.9975


[<KerasTensor: shape=(None, 20, 20, 2) dtype=float32 (created by layer 'environment')>,
 <KerasTensor: shape=(None,) dtype=float32 (created by layer 'state')>]

In [56]:
Z=tf.keras.layers.Input(shape=[5],dtype=tf.int32)
X=tf.keras.layers.Input(shape=[],dtype=tf.int32)

In [59]:
tf.one_hot(Z,depth=X)

2023-06-14 06:05:07.512535: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder_1' with dtype int32 and shape [?]
	 [[{{node Placeholder_1}}]]


<KerasTensor: shape=(None, 5, None) dtype=float32 (created by layer 'tf.one_hot_5')>

In [164]:
m=52
A2=tf.cast(tf.random.uniform([batches,m,m]) > p,dtype=tf.float32)
W2=tf.random.uniform([batches,m,m])
G2=tf.stack([A,W],axis=-1)
X2=tf.random.uniform([batches])
gnn_model.predict([G2,X2])

1/1 [==============================] - 0s 150ms/step


[array([[-1.],
        [-1.],
        [-1.],
        [-1.],
        [-1.],
        [-1.],
        [-1.],
        [-1.],
        [-1.],
        [-1.],
        [-1.],
        [-1.],
        [-1.],
        [-1.],
        [-1.],
        [-1.]], dtype=float32),
 array([[0.05368359, 0.04334371, 0.04518632, 0.05064121, 0.04570146,
         0.05143947, 0.04953609, 0.04892602, 0.04699986, 0.05446874,
         0.05139836, 0.04923844, 0.05057517, 0.0522957 , 0.04983887,
         0.04774844, 0.0497077 , 0.05316909, 0.0535295 , 0.05257224],
        [0.04592943, 0.05222778, 0.05133432, 0.05049992, 0.05339243,
         0.04911728, 0.04447116, 0.04827907, 0.04652907, 0.05009769,
         0.04688105, 0.05116192, 0.05011094, 0.04744739, 0.0453846 ,
         0.04875434, 0.05631608, 0.05804124, 0.05481863, 0.04920571],
        [0.04849744, 0.04872611, 0.05237256, 0.0503167 , 0.0478346 ,
         0.05495047, 0.04880298, 0.05106444, 0.05259023, 0.048517  ,
         0.04659727, 0.05748926, 0.05129892, 0.0471

In [558]:
import pandas as pd
import networkx as nx
cora=pd.read_csv("cora/cora.content",sep="\t",header=None)
cora_graph=nx.read_edgelist("cora/cora.cites",nodetype=int)

def id_mapper(cora,cora_graph):
    mapper={}
    cora_mapped=cora.copy()
    for u,v in cora_graph.edges:
        for z in [u,v]:
            if z not in mapper:
                mapper[z]=len(mapper)
    for _,row in cora.iterrows():
        u = row[0]
        if u not in mapper:
            mapper[u]=len(mapper)
    cora_graph_mapped=nx.DiGraph()
    cora_graph_mapped.add_nodes_from(range(len(mapper)))
    for u,v in cora_graph.edges:
        if u not in mapper or v not in mapper:
            raise RuntimeErro(f"{u,v} not in mapper?")
        cora_graph_mapped.add_edge(mapper[u],mapper[v])
    cora_mapped[0]=cora_mapped[0].map(lambda x: mapper[x])
    return cora_mapped,cora_graph_mapped,mapper

    




In [559]:
cora_mapped,graph_mapped,mapper=id_mapper(cora,cora_graph)
adjacency_matrix=nx.adjacency_matrix(graph_mapped,range(len(graph_mapped.nodes)))
features_count=len(cora.columns)-1


I=range(features_count)
cora_input=cora_mapped[I]
cora_target=cora_mapped[features_count]

class_mapper=tf.keras.layers.StringLookup(output_mode="one_hot")
class_mapper.adapt(cora_target)
cora_target=class_mapper(cora_target)

In [566]:
import numpy as np
import queue
class DataLoader:
    def __init__(self,graph,inputs,targets,max_component_size):
        self.graph=graph
        inputs=inputs.to_numpy()
        S=np.zeros([len(graph),inputs.shape[-1]],dtype=np.int32)
        for row in inputs:
            S[row[0],:]=row
        Y=np.zeros([len(graph),targets.shape[-1]])
        targets=targets.numpy()
        for k,row in enumerate(inputs):
            Y[row[0],:]=targets[k,:]
        self.inputs=S
        self.targets=Y
        self.counter=0
        self.connected_components=list(nx.connected_components(self.graph.to_undirected()))
        self.max_component_size=max_component_size
        self.adjacency_matrix=nx.adjacency_matrix(self.graph,nodelist=range(len(self.graph))).todense()
    def __iter__(self):
        return self
    
    def __next__(self):
        Q=queue.Queue()
        Q.put(np.random.randint(len(self.graph)))
        S=set()
        while not Q.empty():
            u=Q.get()
            for v in self.graph.succ[u]:
                if len(S) == self.max_component_size:
                    break
                if v not in S:
                    S.add(v)
                    Q.put(v)
            if len(S) == self.max_component_size:
                break
        indexes=list(S)
        if len(indexes) < self.max_component_size:
            indexes = indexes+list(np.random.choice(len(self.graph),self.max_component_size-len(indexes)))
        self.counter+=1
        A=self.adjacency_matrix[np.ix_(indexes,indexes)]
        inputs=self.inputs[indexes,:]
        return ((A,inputs),self.targets[indexes,:])

    
    def __len__(self):
        return len(self.graph)
data_loader=DataLoader(graph_mapped,cora_input,cora_target,max_component_size=50)
signature=(  (tf.TensorSpec(shape=(None,None)),tf.TensorSpec(shape=(None,1434))),  tf.TensorSpec(shape=(None,8))  )
dataset=tf.data.Dataset.from_generator(lambda :data_loader,output_signature=signature).batch(32)
for (A0,A1),A2 in dataset:
    print(A2.shape)
    break
    
validation_dataset=tf.data.Dataset.from_generator(lambda :data_loader,output_signature=signature).batch(32).take(64)

(32, 50, 8)


2023-06-14 23:29:23.145116: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


In [564]:
nodes=len(mapper)
x=tf.keras.layers.Input(shape=(None,1434),name="input")
g=tf.keras.layers.Input(shape=(None,None),name="graph")
y=gnn.gcn.WGCN(24,activation="relu",name="H")((g,g,x))
y=gnn.gcn.WGCN(32,activation="relu",name="B")((g,g,y))
output=y
output=tf.keras.layers.Dense(8,activation="softmax",name="prediction")(output)
model=tf.keras.Model(inputs=[g,x],outputs=output)
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),optimizer="adam",metrics=["accuracy"])

In [568]:
model.fit(dataset,epochs=12,steps_per_epoch=84,validation_data=validation_dataset)

Epoch 1/12
84/84 [==============================] - 5s 58ms/step - loss: 0.3832 - accuracy: 0.9254 - val_loss: 0.2036 - val_accuracy: 0.9667
Epoch 2/12
84/84 [==============================] - 5s 58ms/step - loss: 0.1925 - accuracy: 0.9716 - val_loss: 0.1739 - val_accuracy: 0.9775
Epoch 3/12
84/84 [==============================] - 5s 57ms/step - loss: 0.1590 - accuracy: 0.9764 - val_loss: 0.1418 - val_accuracy: 0.9827
Epoch 4/12
84/84 [==============================] - 5s 58ms/step - loss: 0.1955 - accuracy: 0.9686 - val_loss: 0.1952 - val_accuracy: 0.9714
Epoch 5/12
84/84 [==============================] - 5s 60ms/step - loss: 0.1902 - accuracy: 0.9714 - val_loss: 0.1952 - val_accuracy: 0.9735
Epoch 6/12
84/84 [==============================] - 5s 59ms/step - loss: 0.2386 - accuracy: 0.9593 - val_loss: 0.2189 - val_accuracy: 0.9642
Epoch 7/12
84/84 [==============================] - 4s 46ms/step - loss: 0.1593 - accuracy: 0.9773 - val_loss: 0.1358 - val_accuracy: 0.9816
Epoch 8/12
84

2023-06-14 22:13:23.143277: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2023-06-14 22:13:23.163252: W tensorflow/core/framework/op_kernel.cc:1818] INVALID_ARGUMENT: TypeError: `generator` yielded an element of shape (10,) where an element of shape (None, None) was expected.
Traceback (most recent call last):

  File "/home/ramizouari/anaconda3/envs/StochasticGames/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 267, in __call__
    ret = func(*args)

  File "/home/ramizouari/anaconda3/envs/StochasticGames/lib/python3.10/site-packages/tensorflow/python/autograph/impl/api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "/home/ramizouari/anaconda3/envs/StochasticGames/lib/python3.10/site-packages/

InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_3_device_/job:localhost/replica:0/task:0/device:CPU:0}} TypeError: `generator` yielded an element of shape (10,) where an element of shape (None, None) was expected.
Traceback (most recent call last):

  File "/home/ramizouari/anaconda3/envs/StochasticGames/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 267, in __call__
    ret = func(*args)

  File "/home/ramizouari/anaconda3/envs/StochasticGames/lib/python3.10/site-packages/tensorflow/python/autograph/impl/api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "/home/ramizouari/anaconda3/envs/StochasticGames/lib/python3.10/site-packages/tensorflow/python/data/ops/from_generator_op.py", line 235, in generator_py_func
    raise TypeError(

TypeError: `generator` yielded an element of shape (10,) where an element of shape (None, None) was expected.


	 [[{{node PyFunc}}]] [Op:IteratorGetNext]

5416

In [374]:
for k in da

5416

InvalidIndexError: (array([2411, 2125,  824, 1762,  775,  237, 2429,  858, 1378,  323,  179,
        824, 2358, 1093,  579, 2242, 1281, 2453, 2653, 2468, 1593,  752,
       2141, 1326, 1263, 1198, 1523, 1744, 1148, 1114,  167,  730, 1103,
       1385, 2292, 2469, 2223, 1693, 1479, 1645, 1539, 2158, 1414,  539,
       1288, 2065, 2517, 1931, 1833, 1023]), slice(None, None, None))

In [588]:
tf.keras.layers.Dot(axes=(1,1))((x,x))

<KerasTensor: shape=(None, 1434, 1434) dtype=float32 (created by layer 'dot_4')>

In [689]:
R=tf.concat([tf.random.uniform([5,1,8,7])]*8,axis=1)
O=tf.random.uniform([5,1,8,7])

In [690]:
OO=tf.linalg.matmul(R,O,transpose_a=True)

In [693]:
OO[:,2,:,:]

<tf.Tensor: shape=(5, 7, 7), dtype=float32, numpy=
array([[[1.7621603 , 2.2894728 , 1.2296274 , 1.6020095 , 2.2495463 ,
         1.6082793 , 1.8895717 ],
        [2.5168889 , 3.0804372 , 2.3296845 , 3.0225103 , 3.9840174 ,
         2.5132146 , 2.8795948 ],
        [2.0061388 , 3.3219666 , 1.79297   , 2.1187174 , 3.1123812 ,
         2.1617856 , 2.6272242 ],
        [0.7515287 , 1.7849993 , 0.68653375, 0.9826304 , 1.6982837 ,
         1.2125562 , 1.8696753 ],
        [2.1512322 , 2.5824826 , 1.389887  , 2.3883047 , 3.697256  ,
         2.1450052 , 2.6422868 ],
        [1.9249227 , 2.2102168 , 1.6618285 , 2.3116577 , 2.8948588 ,
         2.09665   , 1.90902   ],
        [1.0622282 , 2.2239196 , 1.2219819 , 1.4968036 , 2.1375127 ,
         1.480628  , 1.998343  ]],

       [[2.3857453 , 1.6308819 , 2.0737286 , 1.1093215 , 1.7710793 ,
         1.9981996 , 2.5451553 ],
        [1.3533068 , 0.73741657, 1.5321262 , 0.45784006, 1.2796782 ,
         0.92517835, 1.5147383 ],
        [1.6755888 ,

# 